# AbstractGEN: PyTorch Seq2seq

이 포스트는 6개로 이루어진 "AbstractGEN"의 4번째 포스트입니다. 

이전의 포스트는 정확하게 이야기하면 title을 입력으로 받아 $\rightarrow$ abstract을 생성한다고 이야기하기 어렵습니다. 

따라서 이번 포스트에서는 입력과 출력의 관계를 구분하여 모델을 구현해보겠습니다. 
최종적인 아이디어는 title-abstract을 seq2seq으로 구분하고, abstract을 만드는 decoder에 LM을 또 적용하는 것입니다. 

앞으로의 포스트는 다음과 같습니다:
- #0. AbstractGEN: DataAnalysis
- #1. AbstractGEN: Char-level RNN
- #2. AbstractGEN: Word-level RNN
- **#3. AbstractGEN: Seq2seq**
- #4. AbstractGEN: Seq2seq with attention+teaching force
- #5. AbstractGEN: Applying gpt-2 


In [0]:
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader, Dataset

import pandas as pd
import numpy as np

import re
import collections
import itertools

import tqdm

In [0]:
# 파일을 불러와서 원하는 만큼 잘라, 새로 파일을 만들기 

with open('/content/titles_and_abstracts.txt','r') as f:
  data = f.read()

data = [i for i in data.split('\n') if i != '']
data_dict = []
for i in range(0, len(data), 2):
  data_i = []
  data_i.append(data[i])
  data_i.append(data[i+1])
  data_dict.append(data_i)

df = pd.DataFrame(data_dict[:10000])
df.to_csv('abstract.txt', sep='\t', header=False, index=False)

In [0]:
df.head()

,0,1
0,evaluation technology from speaker identificat...,we propose a multi-step system for the analysi...
1,acquisition of semantic classes for adjectives...,"in this paper , we present a clustering experi..."
2,features for detecting hedge cues,we present a sequential labeling approach to h...
3,semantic extraction with wide-coverage lexical...,we report on results of combining graphical mo...
4,evaluating a crosslinguistic grammar resource ...,this paper evaluates the lingo grammar matrix ...


In [0]:
# cuda gpu 환경이 사용 가능한지 체크

print(torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


## 전처리

In [0]:
# re 모듈을 통해 삭제할 목록들을 정리

FILTERS = "([~,!?\"':;)(])"
SHIFTS = re.compile("([?!.])")
CHANGE_FILTER = re.compile(FILTERS)

In [0]:
def normalize(text):
  text = text.lower()
  text = CHANGE_FILTER.sub("", text)
  text = re.sub('[0-9]+', "", text)
  text = SHIFTS.sub(r" \1", text)
  return text

def parse_line(line):
  line = normalize(line.strip())
  src, trg = line.split('\t')
  # print("title:", src)
  # print("abstract:", trg)
  src_tokens = src.strip().split()
  trg_tokens = trg.strip().split()
  #print(src_tokens)
  return src_tokens, trg_tokens

def build_vocab(words): 
  counts = collections.Counter(words)
  sort_words = sorted(counts.items(),
                      key=lambda x:x[1], reverse = True)
  word_list = ["<UNK>", "<SOS>", "<EOS>"]+ [x[0] for x in sort_words]
  word_dict = dict((w,i) for i,w in enumerate(word_list))
  return word_list, word_dict

def word2tensor(words, word_dict, max_len, padding = 0):
  #  print(words)
  words = words+['<EOS>']
  words = [word_dict.get(w,0) for w in words]
 
 # print(words)
  seq_len = len(words)
  if seq_len < max_len+1:
    words += [padding]*(max_len+1 -seq_len)
  return torch.tensor(words, dtype=torch.int64), seq_len

## DataLoader

In [0]:
class PairDataset(Dataset):
  def __init__(self, path, max_len=300):
    def filter_pair(p):
      return not (len(p[0]) > max_len or len(p[1]) > max_len)

    with open(path) as fp:
      pairs = map(parse_line, fp)
      pairs = filter(filter_pair, pairs)
      pairs = list(pairs)
    
    src = [p[0] for p in pairs]
    trg = [p[1] for p in pairs]

    self.src_word_list, self.src_word_dict = build_vocab(
        itertools.chain.from_iterable(src)
    )
    self.trg_word_list, self.trg_word_dict = build_vocab(
        itertools.chain.from_iterable(trg)
    )

    self.src_data = [word2tensor(
        words, self.src_word_dict, max_len) for words in src]
    self.trg_data = [word2tensor(
        words, self.trg_word_dict, max_len) for words in trg]
  
  def __len__(self):
      return len(self.src_data)
      
  def __getitem__(self, idx):
      src, lsrc = self.src_data[idx]
      trg, ltrg = self.trg_data[idx]
      return src, lsrc, trg, ltrg

In [0]:
batch_size = 64
max_len = 300
path = "/content/abstract.txt"
ds = PairDataset(path, max_len=max_len)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=4)

## Encoder

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim=50, hid_dim=50, n_layers=1, dropout=0.2):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, 
                                      emb_dim, 
                                      padding_idx=0)
        self.lstm = nn.LSTM(emb_dim, 
                            hid_dim, 
                            n_layers,
                            batch_first = True,
                            dropout = dropout)
        
    def forward(self,x, h0=None, l = None):
        x = self.embedding(x)
        if l is not None:
          x = nn.utils.rnn.pack_padded_sequence(x,l,batch_first=True)

        outputs, h = self.lstm(x, h0)
      
        #outputs are always from the top hidden layer
        return h

## Decoder

In [0]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim=50, hid_dim=50, n_layers=1, dropout=0.2):
        super().__init__()
        
        self.embedding = nn.Embedding(output_dim, 
                                      emb_dim, 
                                      padding_idx = 0)
        self.lstm = nn.LSTM(emb_dim, 
                            hid_dim, 
                            n_layers, 
                            batch_first = True,
                            dropout = dropout)
        
        self.linear = nn.Linear(hid_dim, output_dim)
        
    def forward(self, x, h,l=None):
        x = self.embedding(x)
        if l is not None:
          x = nn.utils.rnn.pack_padded_sequence(x,l,batch_first=True)
        x,h = self.lstm(x,h)
        if l is not None:
          x = nn.utils.rnn.pad_packed_sequence(x,batch_first=True)[0]
        
        prediction = self.linear(x)

        
        return prediction, h

### 문장을 생성해줄 함수 정의

In [0]:
def generateSEN(input_string, enc, dec, max_len=140, device='cpu'):
  words = normalize(input_string).split()
  input_tensor, seq_len = word2tensor(words, ds.src_word_dict, max_len)

  input_tensor=input_tensor.unsqueeze(0)
  seq_len = [seq_len]

  sos_inputs = torch.tensor(sos, dtype=torch.int64)
  input_tensor = input_tensor.to(device)
  sos_inputs = sos_inputs.to(device)

  ctx = enc(input_tensor, l=seq_len)
  
  # 시작 토큰과 콘텍스트 벡터를 디코더의 초기값으로!
  z = sos_inputs
  h = ctx
  results = []
  for i in range(max_len):
    o,h = dec(z.view(1,1),h)
    wi = o.detach().view(-1).max(0)[1]
    if wi.item() == eos:
      break
    results.append(wi.item())
    z= wi
  return " ".join(ds.trg_word_list[i] for i in results)

In [0]:
a = torch.randn(1,1)
print(a.size())
print(a)

torch.Size([1, 1])
tensor([[0.1331]])


In [0]:
# 함수 동작 확인 - 학습을 시키지 않았을 때의 결과

enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100,100,2)
generateSEN("the benefit of stochastic pp attachment to a rule-based parser", enc, dec, 140,"cpu")

'barge-in kyotoebmt kyotoebmt inuktitut clear-cut kyotoebmt supercomputer lgpl conjecture gist-it courses kyotoebmt aggregated analysers advantages higherorder conjecture fluctuations mouth mouth reliably mouth mouth childes mouth mouth learning/nlp mouth cutaneous conjecture mouth mouth grapples grapples grapples grapples grapples grapples grapples incident exclude concrete concrete declared kyotoebmt analysers icd--cm gist-it cutaneous conjecture conjecture conjecture inuktitut inuktitut inuktitut turkic happens characterization formation formation sensory-motor class-conditional sensory-motor setups defense lgpl lgpl class-conditional aspossible aspossible kyotoebmt aspossible adaboost adaboost agglomeratively kyotoebmt agglomeratively disinformation wit courses subphrase superfluous grapples grapples grapples grapples grapples grapples spring spring spring gist-it hapaxlegomena grapples grapples grapples grapples grapples grapples spring spring spring gist-it hapaxlegomena grapples

# 모델 학습

In [0]:
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100,100,2)
enc.to("cuda:0")
dec.to("cuda:0")

opt_enc = optim.Adam(enc.parameters(), 0.002)
opt_dec = optim.Adam(dec.parameters(), 0.002)

loss_f = nn.CrossEntropyLoss()

In [0]:
print(ds.trg_word_dict)
print(ds.src_word_dict)

{'<UNK>': 0, '<SOS>': 1, '<EOS>': 2, 'the': 3, '.': 4, 'of': 5, 'a': 6, 'and': 7, 'to': 8, 'in': 9, 'we': 10, 'for': 11, 'that': 12, 'is': 13, 'on': 14, 'this': 15, 'with': 16, 'as': 17, 'are': 18, 'an': 19, 'our': 20, 'by': 21, 'from': 22, 'paper': 23, 'system': 24, 'model': 25, 'which': 26, 'language': 27, 'using': 28, 'be': 29, 'can': 30, 'data': 31, 'results': 32, 'approach': 33, 'task': 34, 'translation': 35, 'method': 36, 'word': 37, 'information': 38, 'show': 39, 'based': 40, '%': 41, 'it': 42, 'features': 43, 'present': 44, 'semantic': 45, 'models': 46, 'corpus': 47, 'text': 48, 'two': 49, 'used': 50, 'these': 51, 'use': 52, 'such': 53, 'performance': 54, 'systems': 55, 'learning': 56, 'or': 57, 'different': 58, 'words': 59, 'have': 60, 'not': 61, 'between': 62, 'also': 63, 'machine': 64, 'has': 65, 'new': 66, 'methods': 67, 'evaluation': 68, 'both': 69, 'algorithm': 70, 'set': 71, 'training': 72, 'more': 73, 'into': 74, 'over': 75, 'propose': 76, 'analysis': 77, 'their': 78, '

In [0]:
print(len(ds.src_data))

9999


In [0]:
print(ds.src_data[0])
print(len(ds.src_data))
p = []
for (a,b) in ds.src_data:
  p.append(b)
print(max(p))
print(p)

(tensor([  37,   95,   16, 1571,   75,   10, 1254,   31,    6, 2596,   28,    3,
         594, 1406, 1143,    2,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,  

In [0]:
o = []
for (a,b) in ds.trg_data:
  o.append(b)
  w = len(a)
print(max(o))
print(w)

292
301


In [0]:
def to2D(x):
  shapes = x.shape
  return x.reshape(shapes[0] * shapes[1], -1)

for epoch in range(1):
  enc.train(), dec.train()
  losses = []

  for x, lx, y, ly in tqdm.tqdm(loader):
    # x의 PackedSequence를 만들기 위해 번역 소스의 길이로 내림차순 정렬한다.
    lx, sort_idx = lx.sort(descending=True)
    x,y,ly = x[sort_idx], y[sort_idx], ly[sort_idx]
    x,y = x.to("cuda:0"), y.to("cuda:0")

    ctx = enc(x,l=lx)

    # y의 PackedSequence를 만들기 위해 번역 소스의 길이로 내림차순 정렬한다.
    ly, sort_idx = ly.sort(descending=True)
    y  = y[sort_idx]

    
    print("ctx shape:", ctx[0].shape)
    print()
    h0 = (ctx[0][:, sort_idx,:], ctx[1][:, sort_idx, :])

    z = y[:,:-1].detach()
    z[z==-100] = 0

    o,_ = dec(z, h0, l=ly-1)

    loss = loss_f(to2D(o[:]), to2D(y[:,1:max(ly)]).squeeze())

    enc.zero_grad(), dec.zero_grad()
    loss.backward()

    opt_enc.step(), opt_dec.step()
    losses.append(loss.item())

  enc.eval(), dec.eval()
  print(epoch)
  with torch.no_grad():
    sen = "cross-document coreference on a large scale corpus"
    print(sen)
    print(generateSEN(sen, enc, dec, max_len=140, device='cuda:0'))

  0%|          | 0/157 [00:00<?, ?it/s]

ctx shape: torch.Size([2, 64, 100])



  1%|          | 1/157 [00:00<01:42,  1.53it/s]

ctx shape: torch.Size([2, 64, 100])



  1%|▏         | 2/157 [00:01<01:36,  1.60it/s]

...

ctx shape: torch.Size([2, 64, 100])



 99%|█████████▊| 155/157 [01:06<00:00,  2.33it/s]

ctx shape: torch.Size([2, 64, 100])



100%|██████████| 157/157 [01:07<00:00,  2.32it/s]


ctx shape: torch.Size([2, 15, 100])

0
cross-document coreference on a large scale corpus
paper paper a a a the the the the the the the of of of of of the the the . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


당연한 이야기겠지만, 기계번역과 같은 방식은 논문의 초록을 생성하는데 적합하지 않았다. 
